In [18]:
import joblib
import pandas as pd
import numpy as np
# File path where the model is stored
loaded_rf = joblib.load(r"C:\Users\me513\Documents\sc\gp\random_forest.joblib")

# Now you can use the loaded model for predictions
# For example:
# prediction = random_forest_model.predict(X_test)


In [19]:
df1 = pd.read_excel(r"C:\Users\me513\Documents\sc\gp\Copy of شدة_الاعراض(1).xlsx")
discrp = pd.read_excel(r"C:\Users\me513\Documents\sc\gp\symptom_Description.xlsx")
ektra7at = pd.read_excel(r"C:\Users\me513\Documents\sc\gp\symptom_precaution.xlsx")
df1['علامة مرض'] = df1['علامة مرض'].str.replace('_',' ')

In [20]:
def predict(x,S1=' ',S2=' ',S3=' ',S4=' ',S5=' ',S6=' ',S7=' ',S8=' ',S9=' ',S10=' ',S11=' ',S12=' ',S13=' ',S14=' ',S15=' ',S16=' ',S17=' '):
    psymptoms = [S1,S2,S3,S4,S5,S6,S7,S8,S9,S10,S11,S12,S13,S14,S15,S16,S17]
    #print(psymptoms)
    a = np.array(df1["علامة مرض"])
    b = np.array(df1["وزن"])
    for j in range(len(psymptoms)):
        for k in range(len(a)):
            if psymptoms[j]==a[k]:
                psymptoms[j]=b[k]
            if psymptoms[j]==' ':
                psymptoms[j]=0
    psy = [psymptoms]
    pred2 = x.predict(psy)
    disp= discrp[discrp['Disease']==pred2[0]]
    disp = disp.values[0][1]
    recomnd = ektra7at[ektra7at['Disease']==pred2[0]]
    c=np.where(ektra7at['Disease']==pred2[0])[0][0]
    precuation_list=[]
    for i in range(1,len(ektra7at.iloc[c])):
          precuation_list.append(ektra7at.iloc[c,i])
    print("The Disease Name: ",pred2[0])
    print("The Disease Discription: ",disp)
    print("Recommended Things to do at home: ")
    for i in precuation_list:
        print(i)
    return pred2 ,disp,precuation_list

In [26]:
import json
import pandas as pd 
from tensorflow.keras.models import load_model
from flask import Flask, request, Response
import numpy as np
import tensorflow as tf
from flask import Flask, request, render_template, jsonify

app = Flask(__name__)

# Custom encoder function to handle ndarray objects
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

# Define your function to make predictions
def respond(S1=' ',S2=' ',S3=' ',S4=' ',S5=' ',S6=' ',S7=' ',S8=' ',S9=' ',S10=' ',S11=' ',S12=' ',S13=' ',S14=' ',S15=' ',S16=' ',S17=' '):
    # Assuming you have a function called predict to make predictions
    name, description, precaution_list = predict(loaded_rf,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10,S11,S12,S13,S14,S15,S16,S17)
    return name, description, precaution_list

# Define a route for your API
@app.route('/predictAPI', methods=['POST'])
def create_page():
    # Assuming you have input_sentence available in the request
    #input_sentence = request.json.get('inputs', [])
    request_data = request.json
    resp={'S1':'','S2':'','S3':'','S4':'','S5':' ','S6':' ','S7':' ','S8':' ','S9':' ','S10':' ','S11':' ','S12':' ','S13':' ','S14':' ','S15':' ','S16':' ','S17':' '}
    for i in range(0,17):
        value = request_data.get(f'S{i+1}')
        if value is not None:
            resp[f'S{i+1}'] = request_data.get(f'S{i+1}')
        else:
            continue
        
    name, description, precaution_list = respond(**resp)
    
    # Use json.dumps with custom encoder to ensure serialization of ndarray
    response_data = {'name': name, 'description': description, 'precaution_list': precaution_list}
    return Response(json.dumps(response_data, cls=NumpyEncoder), mimetype='application/json')

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.60.142:5000
Press CTRL+C to quit
192.168.60.142 - - [24/Apr/2024 22:11:54] "POST /predictAPI HTTP/1.1" 200 -


The Disease Name:  تلوث فطري
The Disease Discription:  تحدث الالتهابات الفطرية عند البشر عندما تسيطر الفطريات الغازية على منطقة من الجسم وتكون أكثر من قدرة الجهاز المناعي على التعامل معها. يمكن للفطريات أن تعيش في الهواء والتربة والماء والنباتات. كما أن هناك بعض الفطريات التي تعيش بشكل طبيعي في جسم الإنسان. مثل العديد من الميكروبات، هناك فطريات مفيدة وفطريات ضارة.
Recommended Things to do at home: 
حمام مرتين
استخدمي الديتول أو النيم في ماء الاستحمام
إبقاء المنطقة المصابة جافة
استخدام الملابس النظيفة


[2024-04-24 22:13:20,935] ERROR in app: Exception on /predictAPI [POST]
Traceback (most recent call last):
  File "c:\Users\me513\anaconda3\envs\gp\lib\site-packages\flask\app.py", line 1463, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\me513\anaconda3\envs\gp\lib\site-packages\flask\app.py", line 872, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\me513\anaconda3\envs\gp\lib\site-packages\flask\app.py", line 870, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\me513\anaconda3\envs\gp\lib\site-packages\flask\app.py", line 855, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "C:\Users\me513\AppData\Local\Temp\ipykernel_13944\4050187976.py", line 38, in create_page
    name, description, precaution_list = respond(**resp)
  File "C:\Users\me513\AppData\Local\Temp\ipykernel_13944\4050187976.py", line 21, in respond
 